In [ ]:
import pandas as pd
import numpy as np
import os, cv2

try:
    %env CUDA_DEVICE_ORDER=PCI_BUS_ID
    %env CUDA_VISIBLE_DEVICES=0
except NameError:
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import tensorflow as tf
from tensorflow.keras.models import Model, load_model

import matplotlib.pyplot as plt

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)  # set this TensorFlow session as the default session for Keras

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())  ## to use which GPU in use


In [ ]:
##Loading the model
model = load_model('/home/Erdal.Genc/covid_work/Padchest_work/')

In [ ]:
df_path = '/home/Erdal.Genc/covid_work/PadChest_csv/Padchest_filtered_withPath.csv'
df = pd.read_csv(df_path, low_memory= False)

In [ ]:
# img_ = '/mnt/dsets/ChestXrays/PadChest/image_zips/0/99744230716892055301280916536204938895_oo9nk5.png'
index = 50
label = df['new_labels'][index]
img_ = df['Path'][index]
print(label, '\n', img_)
last_conv_layer_name = 'res5c_branch2c'

In [ ]:

print('tensorflow {}'.format(tf.__version__))

In [ ]:
from PIL import Image
from skimage.transform import resize

_img = Image.open(img_)
img = np.asarray(_img)
dim = (224,224)
img = resize(img, dim)
print(img.shape)
plt.imshow(np.abs(img), cmap = 'gray')
plt.show()
img_3D = np.stack((img,)*3, axis=-1) ## converts 1-Channel to 3-Channel image
img_ready = img_3D.reshape((1, *img_3D.shape))
y_pred = model.predict(img_ready)

In [ ]:
grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(last_conv_layer_name).output, model.output])

with tf.GradientTape() as tape:
    conv_outputs, predictions = grad_model(np.array([img_3D]))
    loss = predictions[:, np.argmax(max(y_pred))] ##getting Class Index


In [ ]:
output = conv_outputs[0]
grads = tape.gradient(loss, conv_outputs)[0]

gate_f = tf.cast(output > 0, 'float32')
gate_r = tf.cast(grads > 0, 'float32')
guided_grads = tf.cast(output > 0, 'float32') * tf.cast(grads > 0, 'float32') * grads

weights = tf.reduce_mean(guided_grads, axis=(0, 1))

cam = np.ones(output.shape[0: 2], dtype = np.float32)

for i, w in enumerate(weights):
    cam += w * output[:, :, i]

cam = cv2.resize(cam.numpy(), (224, 224))
cam = np.maximum(cam, 0)
heatmap = (cam - cam.min()) / (cam.max() - cam.min())

cam = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)

output_image = cv2.addWeighted(cv2.cvtColor(img.astype('uint8'), cv2.COLOR_RGB2BGR), 0.5, cam, 1, 0)

plt.imshow(output_image)

In [ ]:
print(np.argmax(max(y_pred)))